# Estratégias de treino

Até agora no curso, vimos somente uma estratégia de treino para redes neurais: o treinamento do zero.
Entretanto, há outras formas de se explorar redes neurais.
Nessa aula, vamos rever a estratégia treinamento do zero além de apresentar duas novas formas:

1.   rede neural como um extrator de características, e
2.   *fine-tuning*.

Para cada uma dessas estratégias, vamos apresentar sua definição, vantagens e desvantagens.


## Configuração do ambiente

In [ ]:
import time, os, sys, numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torchinfo import summary

import time, os, sys, numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n = torch.cuda.device_count()
devices_ids= list(range(n))
print(device)

## Carregamento das bases de dados

A função `load_data_cifar10` carrega e prepara o dataset CIFAR-10 para treinamento e teste.

In [ ]:
def load_data_cifar10(batch_size, resize=None, root='/pgeoprj2/ciag2024/dados/'):
    root = os.path.expanduser(root)

    transformer = []
    if resize:
        transformer += [torchvision.transforms.Resize(resize)]
    transformer += [torchvision.transforms.ToTensor()]
    transformer = torchvision.transforms.Compose(transformer)

    cifar10_train = torchvision.datasets.CIFAR10(root=root, train=True, transform=transformer)
    cifar10_test = torchvision.datasets.CIFAR10(root=root, train=False, transform=transformer)

    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = torch.utils.data.DataLoader(cifar10_train,
                                            batch_size, shuffle=True,
                                            num_workers=num_workers)

    test_iter = torch.utils.data.DataLoader(cifar10_test,
                                            batch_size, shuffle=False,
                                            num_workers=num_workers)
    return train_iter, test_iter

## Funções auxiliares

* `evaluate_accuracy` calcula a acurácia de um modelo em um dataset.

* `train_validate` implemneta o treinamento e validação de uma rede.

In [ ]:
def evaluate_accuracy(data_iter, net, loss):
    acc_sum, n, l = torch.Tensor([0]), 0, 0
    net.eval()

    with torch.no_grad():
      for X, y in data_iter:
          X, y = X.to(device), y.to(device)
          y_hat = net(X)
          l += loss(y_hat, y).sum()
          acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
          n += y.size()[0]

    return acc_sum.item() / n, l.item() / len(data_iter)

def train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs):
    print('training on', device)

    for epoch in range(num_epochs):
        net.train()
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()

        for X, y in train_iter:
            X,  y = X.to(device), y.to(device)
            y_hat = net(X)

            trainer.zero_grad()
            l = loss(y_hat, y).sum()

            l.backward()
            trainer.step()

            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.size()[0]

        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss)

        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / len(train_iter), train_acc_sum / n, test_loss,
                 test_acc, time.time() - start))

## Treinamento do zero

Como dito anteriormente, essa foi a única estratégia vista até o momento no curso.
Nessa estratégia, uma rede neural é proposta, **inicializada com pesos aleatórios** e treinada até convergir.
A **vantagem** dessa estratégia é liberdade para definir como quiser a arquitetura da rede e seus hiper-parâmetros
Por outro lado, a **desvantagem** é que essa estratégia requer muitos dados para convergir a rede inicializada aleatoriamente.
Logo, se tivermos poucos dados, essa não é a estratégia mais recomendada.
Abaixo, uma representação visual dessa estratégia.

<p align="center">
  <img width=600 src="https://drive.google.com/uc?export=view&id=1_bBQjyoDqB3kQMncmVkuJwSxDs3rqUmM">
</p>

Apesar de já termos visto essa estratégia na prática, vamos vê-la aqui novamente para efeitos de comparação com as outras técnicas. Para tal, vamos, primeiro, definimos a arquitetura da [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf).



In [ ]:
class AlexNet(nn.Module):
    def __init__(self, input_channels, classes=10, **kwargs):
        super(AlexNet, self).__init__(**kwargs)
        self.convs = nn.Sequential(
            nn.Conv2d(in_channels=input_channels, out_channels=96, kernel_size=11, stride=4, padding=0),   # entrada: (b, 3, 227, 227) e saida: (b, 96, 55, 55)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),                                   # entrada: (b, 96, 55, 55) e saida: (b, 96, 27, 27)

            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2),  # entrada: (b, 96, 27, 27) e saida: (b, 256, 27, 27)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),                                   # entrada: (b, 256, 27, 27) e saida: (b, 256, 13, 13)
            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1), # entrada: (b, 256, 13, 13) e saida: (b, 384, 13, 13)
            nn.ReLU(),
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1), # entrada: (b, 384, 13, 13) e saida: (b, 384, 13, 13)
            nn.ReLU(),
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1), # entrada: (b, 384, 13, 13) e saida: (b, 256, 13, 13)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0)                                    # entrada: (b, 256, 13, 13) e saida: (b, 256, 6, 6)
        )

        self.features = nn.Sequential(
            nn.Flatten(),                                                                     # entrada: (b, 256, 13, 13) e saida: (b, 256*6*6) = (b, 9216)
            nn.Linear(9216, 4096),                                                             # entrada: (b, 9216) e saida: (b, 4096)
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),                                                             # entrada: (b, 4096) e saida: (b, 4096)
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, classes)                                                          # entrada: (b, 4096) e saida: (b, 10)
        )

    def forward(self, x):
        x = self.convs(x)
        x = self.features(x)
        return x

In [ ]:
num_epochs, lr, batch_size, wd_lambda = 20, 0.01, 100, 0.0001

net = AlexNet(3, 10)
net.to(device)
print(summary(net, (batch_size, 3, 227, 227)))

loss = nn.CrossEntropyLoss()

train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

trainer = optim.SGD(net.parameters(), lr=lr, weight_decay=wd_lambda, momentum=0.9)

train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)

É muito comum se usar redes já existentes para aprender características em novos dados.
Por isso, muitos frameworks já deixam as arquiteturas mais famosas pré-implementadas para que possam ser usadas.

No Pytorch, podemos importar uma rede [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) usando o pacote [torchvision.models](https://pytorch.org/docs/stable/torchvision/models.html#torchvision-models) do Pytorch.
Há várias arquiteturas pré-definidas nessa biblioteca, incluindo várias [DenseNets](https://arxiv.org/pdf/1608.06993.pdf) e [ResNets](https://arxiv.org/abs/1603.05027), [VGGs](https://arxiv.org/abs/1409.1556), [SqueezeNets](https://arxiv.org/abs/1602.07360), etc.

In [ ]:
num_epochs, lr, batch_size, wd_lambda = 20, 0.01, 100, 0.0001

# Rede importada do PyTorch
net = torchvision.models.alexnet(num_classes=10)
net.to(device)
print(summary(net, (batch_size, 3, 227, 227)))

loss = nn.CrossEntropyLoss()

train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

trainer = optim.SGD(net.parameters(), lr=lr, weight_decay=wd_lambda, momentum=0.9)

train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)

## Extrator de características

A terceira e última estratégia, mostrada na figura abaixo, é usar uma rede neural pré-treinada em algum dataset grande para extrair características de um outro dataset. Essa estratégia é preferível quando o dataset que se quer extrair as *features* tem muito poucas amostras, inviabilizando o treinamento ou *fine-tuning* da rede.

<p align="center">
  <img width=600 src="https://drive.google.com/uc?export=view&id=1pWGfQIAeOODIvm-IQ7De4kl60XpRYbb5">
</p>

Existem duas formas de se explorar essa estratégia. A primeira consiste em substituir e treinar somente a última camada da rede neural. Nessa primeira forma, todas as outras camadas da rede ficam com *learning rate* 0, ou seja, não aprendem nada, e são somente usadas como codificadores/extratores de características. A segunda forma, *features* das imagens do dataset que se quer classificar são extraídas da penúltima camada da rede pré-treinada (geralmente, a camada antes da camada de classificação). Essas *features* são então usadas para se treinar um agoritmo externo (como um SVM ou *random forest*), que então classifica o dataset.

In [ ]:
net = torchvision.models.alexnet(pretrained=True)

for param in net.parameters():
    param.requires_grad = False

print(summary(net, (batch_size, 3, 227, 227)))

num_ftrs = net.classifier[6].in_features
net.classifier[6] = nn.Linear(num_ftrs,10) # Alterando a última layer para retornar 10 classes ao invés de 1000

net.to(device)

# Verifique no output a última camada do classifier, podemos ver que sua saída é 10
print(net)

# Podemos ver que este output mostra que apenas  40970 parâmetros serão treinados. Ou seja, somente a última camada.
print(summary(net, (batch_size, 3, 227, 227)))

# Código retirado de https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

# Veja os parâmetros a serem otimizados/atualizados nesta execução.
# Se estivermos fazendo finetuning, atualizaremos todos os parâmetros.
# No entanto, se estivermos fazendo o método feature extract, atualizaremos apenas os parâmetros que acabamos de inicializar, ou seja, os parâmetros com require_grad como True.
print("Params to learn: ")

params_to_update = []
for name,param in net.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

In [ ]:
# Treinando a última camada da rede acima
num_epochs, lr, batch_size, wd_lambda = 20, 0.001, 100, 0.0001

loss = nn.CrossEntropyLoss()

train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

trainer = optim.SGD(params_to_update, lr=lr, weight_decay=wd_lambda, momentum=0.9)

train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)

In [ ]:
num_epochs, lr, batch_size, wd_lambda = 20, 0.01, 100, 0.0001

net = torchvision.models.alexnet(pretrained=True)
net.to(device)

train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# Remover a última camada fully-connected
new_classifier = nn.Sequential(*list(net.classifier.children())[:-1])
net.classifier = new_classifier

print(summary(net, (batch_size, 3, 227, 227)))

first = True
with torch.no_grad():
    for X, y in train_iter:
        X, y = X.to(device), y.to(device)
        features = net(X)
        if first is True:
          train_features = features.cpu().numpy()
          train_labels = y.cpu().numpy()
          first = False
        else:
          train_features = np.concatenate((train_features, features.cpu().numpy()))
          train_labels = np.concatenate((train_labels, y.cpu().numpy()))

first = True
with torch.no_grad():
    for X, y in test_iter:
        X, y = X.to(device), y.to(device)
        features = net(X)
        if first is True:
          test_features = features.cpu().numpy()
          test_labels = y.cpu().numpy()
          first = False
        else:
          test_features = np.concatenate((test_features, features.cpu().numpy()))
          test_labels = np.concatenate((test_labels, y.cpu().numpy()))

print(train_features.shape, test_features.shape)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

clf = LinearSVC()
clf.fit(train_features, train_labels)

pred = clf.predict(test_features)
print(accuracy_score(test_labels, pred))

## *Fine-tuning*

A segunda estratégia é chamada de *fine-tuning*, e é comumente classificada como um estratégia de *transfer learning*, onde o aprendizado é transferido entre datasets.
Especificamente, esta estratégia, representada na figura abaixo, tenta usar um modelo pré-treinado aprendido anteriormente em algum dataset (geralmente muito grande, como o [ImageNet](http://www.image-net.org/)) para classificar outro conjunto de dados diferentes (geralmente com poucas amostras).

<p align="center">
  <img width=600 src="https://drive.google.com/uc?export=view&id=1CoOfpMcQAEl9YAL0lgW11LLYpDcnL4dQ">
</p>

Como esses dados podem possuir características diferentes, treinamos a rede usando um *learning rate* pequeno, apenas para fazer pequenos ajustes nos pesos. Entretanto, como esses datasets geralmente tem número e classes diferentes, a última camada não é usada nessa transferência de peso e, geralmente, é inicializada aleatoriamente (e por isso, tem um *learning rate* mais alto que as demais camadas).

Por fim, é um [fato conhecido](https://arxiv.org/pdf/1602.01517.pdf) que as redes neurais conseguem aprender características de baixo nível nas camadas iniciais. Geralmente, essas características são comuns à vários datasets. Por isso, uma opção durante o processo de *fine-tuning* é "congelar" as camadas iniciais (ou seja, não treiná-las) e treinar somente as demais camadas com taxa de aprendizado bem pequeno (exceto pela camada de classificação).

No bloco de código abaixo, importamos a rede pré-treinada [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), que foi treinada no dataset do [ImageNet](http://www.image-net.org/), que tem 1000 classes. Como iremos fazer *fine-tuning* nessa arquitetura para o dataset do [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html), que tem somente 10 classes, removeremos a última camada e criaremos uma nova camada inicializada aleatoriamente.

In [ ]:
net = torchvision.models.alexnet(pretrained=True)

print(summary(net, (batch_size, 3, 227, 227)))

num_ftrs = net.classifier[6].in_features
net.classifier[6] = nn.Linear(num_ftrs, 10) # Alterando a última layer para retornar 10 classes ao invés de 1000

net.to(device)

# Verifique no output a última camada do classifier, podemos ver que sua saída é 10
print(net)

# Podemos ver que este output mostra que apenas 40970 parâmetros serão treinados. Ou seja, somente a última camada.
print(summary(net, (batch_size, 3, 227, 227)))

num_epochs, lr, batch_size, wd_lambda = 20, 0.001, 100, 0.0001

loss = nn.CrossEntropyLoss()

train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

trainer = optim.SGD([
                {'params': net.features.parameters(), 'lr': lr * 0.1},
                {'params': net.classifier[0:6].parameters(), 'lr': lr * 0.1},
                {'params': net.classifier[6].parameters(), 'lr': lr}], weight_decay=wd_lambda, momentum=0.9)

train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)

## Atividade

1. É possível melhorar o resultado obtido anteriormente?
Estude o [model_zoo](https://pytorch.org/vision/stable/models.html)  e tente usar as estratégias anteriores com diferentes redes neurais para melhorar o resultado.
Algumas redes possíveis:

- [MobileNets](https://arxiv.org/abs/1801.04381)
- [VGGs](https://arxiv.org/abs/1409.1556)
- [ResNets](https://arxiv.org/abs/1603.05027)
- [DenseNets](https://arxiv.org/pdf/1608.06993.pdf)

2. Procure agora congelar algumas camadas para realizar o *fine-tuning*. Essa estratégia é melhor quando se tem poucas imagens para fazer o *fine-tuning*.

3. Procura usar outros algoritmos de aprendizado de máquina (como [*random forest*](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) e [SVM-RBF](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)) para classificar *deep features* extraídas de uma rede neural pré-treinada.

- Procure também extrair e classificar *features* de outras camadas convolucionais.

4. Procure usar as diferentes estratégias para melhorar os resultados dos datasets que já usamos, como [MNIST](https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.MNIST) e [Fashion MNIST](https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.FashionMNIST).